# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Strong: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer. 
  - Boss: Implement [Self-Attention Pooling](https://arxiv.org/pdf/2008.01077v1.pdf) & [Additive Margin Softmax](https://arxiv.org/pdf/1801.05599.pdf) to further boost the performance.

- Other links
  - Competiton: [link](https://www.kaggle.com/t/49ea0c385a974db5919ec67299ba2e6b)
  - Slide: [link](https://docs.google.com/presentation/d/1LDAW0GGrC9B6D7dlNdYzQL6D60-iKgFr/edit?usp=sharing&ouid=104280564485377739218&rtpof=true&sd=true)
  - Data: [link](https://github.com/googly-mingto/ML2023HW4/releases)

# Download dataset
- Data is [here](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

In [ ]:
!wget https://github.com/googly-mingto/ML2023HW4/releases/download/data/Dataset.tar.gz.partaa
!wget https://github.com/googly-mingto/ML2023HW4/releases/download/data/Dataset.tar.gz.partab
!wget https://github.com/googly-mingto/ML2023HW4/releases/download/data/Dataset.tar.gz.partac
!wget https://github.com/googly-mingto/ML2023HW4/releases/download/data/Dataset.tar.gz.partad

!cat Dataset.tar.gz.part* > Dataset.tar.gz
!rm Dataset.tar.gz.partaa
!rm Dataset.tar.gz.partab
!rm Dataset.tar.gz.partac
!rm Dataset.tar.gz.partad
# unzip the file
!tar zxf Dataset.tar.gz
!rm Dataset.tar.gz

In [ ]:
!tar zxf Dataset.tar.gz

# Set seed

In [1]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(87)

# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [2]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
 
 
class myDataset(Dataset):
	def __init__(self, data_dir, segment_len=128):
		self.data_dir = data_dir
		self.segment_len = segment_len
	
		# Load the mapping from speaker neme to their corresponding id. 
		mapping_path = Path(data_dir) / "mapping.json"
		mapping = json.load(mapping_path.open())
		self.speaker2id = mapping["speaker2id"]
	
		# Load metadata of training data.
		metadata_path = Path(data_dir) / "metadata.json"
		metadata = json.load(open(metadata_path))["speakers"]
	
		# Get the total number of speaker.
		self.speaker_num = len(metadata.keys())
		self.data = []
		for speaker in metadata.keys():
			for utterances in metadata[speaker]:
				self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
 
	def __len__(self):
			return len(self.data)
 
	def __getitem__(self, index):
		feat_path, speaker = self.data[index]
		# Load preprocessed mel-spectrogram.
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		# Segmemt mel-spectrogram into "segment_len" frames.
		if len(mel) > self.segment_len:
			# Randomly get the starting point of the segment.
			start = random.randint(0, len(mel) - self.segment_len)
			# Get a segment with "segment_len" frames.
			mel = torch.FloatTensor(mel[start:start+self.segment_len])
		else:
			mel = torch.FloatTensor(mel)
		# Turn the speaker id into long for computing loss later.
		speaker = torch.FloatTensor([speaker]).long()
		return mel, speaker
 
	def get_speaker_number(self):
		return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [3]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
	# Process features within a batch.
	"""Collate a batch of data."""
	mel, speaker = zip(*batch)
	# Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
	mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
	# mel: (batch size, length, 40)
	return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
	"""Generate dataloader"""
	dataset = myDataset(data_dir)
	speaker_num = dataset.get_speaker_number()
	# Split dataset into training dataset and validation dataset
	trainlen = int(0.9 * len(dataset))
	lengths = [trainlen, len(dataset) - trainlen]
	trainset, validset = random_split(dataset, lengths)

	train_loader = DataLoader(
		trainset,
		batch_size=batch_size,
		shuffle=True,
		drop_last=True,
		num_workers=n_workers,
		pin_memory=True,
		collate_fn=collate_batch,
	)
	valid_loader = DataLoader(
		validset,
		batch_size=batch_size,
		num_workers=n_workers,
		drop_last=True,
		pin_memory=True,
		collate_fn=collate_batch,
	)

	return train_loader, valid_loader, speaker_num

In [4]:
dataset = myDataset('data/Dataset')
len(dataset)

56666

# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

### Simple

Acc=0.66025

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Classifier(nn.Module):
	def __init__(self, d_model=80, n_spks=600, dropout=0.1):
		super().__init__()
		# Project the dimension of features from that of input into d_model.
		self.prenet = nn.Linear(40, d_model)
		self.encoder_layer = nn.TransformerEncoderLayer(
			d_model=d_model, dim_feedforward=256, nhead=2
		)
		# self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

		# Project the the dimension of features from d_model into speaker nums.
		self.pred_layer = nn.Sequential(
			nn.Linear(d_model, d_model),
			nn.Sigmoid(),
			nn.Linear(d_model, n_spks),
		)

	def forward(self, mels):
		"""
		args:
			mels: (batch size, length, 40)
		return:
			out: (batch size, n_spks)
		"""
		# input: [B, seq_len, 40]
		out = self.prenet(mels)  # [B, seq_len, d_model]
		out = out.permute(1, 0, 2)  # [seq_len, B, d_model]
		out = self.encoder_layer(out)  # [seq_len, B, d_model]
		out = out.transpose(0, 1)  # [B, seq_len, d_model]
		# mean pooling
		stats = out.mean(dim=1)  # [B, d_model]
		out = self.pred_layer(stats)  # [B, n_spks]
		return out

### Medium

Acc=0.81750

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

# TODO: Modify the parameters of the transformer modules in the sample code
class ClassifierMeidum(nn.Module):
	def __init__(self, d_model=512, n_spks=600, dropout=0.1):
		super().__init__()
		# Project the dimension of features from that of input into d_model.
		self.prenet = nn.Linear(40, d_model)
		self.encoder_layer = nn.TransformerEncoderLayer(
			d_model=d_model, dim_feedforward=1024, nhead=4
		)
		self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=6)

		# Project the the dimension of features from d_model into speaker nums.
		self.pred_layer = nn.Sequential(
			nn.Linear(d_model, d_model),
			nn.Sigmoid(),
			nn.Linear(d_model, n_spks),
		)

	def forward(self, mels):
		"""
		args:
			mels: (batch size, length, 40)
		return:
			out: (batch size, n_spks)
		"""
		# input: [B, seq_len, 40]
		out = self.prenet(mels)  # [B, seq_len, d_model]
		out = out.permute(1, 0, 2)  # [seq_len, B, d_model]
		# out = self.encoder_layer(out)  # [seq_len, B, d_model]
		out = self.encoder(out)
		out = out.transpose(0, 1)  # [B, seq_len, d_model]
		# mean pooling
		stats = out.mean(dim=1)  # [B, d_model]
		out = self.pred_layer(stats)  # [B, n_spks]
		return out

summary(ClassifierMeidum(), (1, 100, 40))

Layer (type:depth-idx)                        Output Shape              Param #
ClassifierMeidum                              [1, 600]                  2,102,784
├─Linear: 1-1                                 [1, 100, 512]             20,992
├─TransformerEncoder: 1-2                     [100, 1, 512]             --
│    └─ModuleList: 2-1                        --                        --
│    │    └─TransformerEncoderLayer: 3-1      [100, 1, 512]             2,102,784
│    │    └─TransformerEncoderLayer: 3-2      [100, 1, 512]             2,102,784
│    │    └─TransformerEncoderLayer: 3-3      [100, 1, 512]             2,102,784
│    │    └─TransformerEncoderLayer: 3-4      [100, 1, 512]             2,102,784
│    │    └─TransformerEncoderLayer: 3-5      [100, 1, 512]             2,102,784
│    │    └─TransformerEncoderLayer: 3-6      [100, 1, 512]             2,102,784
├─Sequential: 1-3                             [1, 600]                  --
│    └─Linear: 2-2                        

### Strong

Acc=0.88500

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from conformer import ConformerBlock

# TODO: Construct Conformer replace Transformer
class ClassifierStrong(nn.Module):
	def __init__(self, d_model=512, n_spks=600, dropout=0.1):
		super().__init__()
		# Project the dimension of features from that of input into d_model.
		self.prenet = nn.Linear(40, d_model)
		self.conformer = ConformerBlock(dim=d_model,
                                            heads=8,
                                            ff_mult=4,
                                            attn_dropout=dropout,
                                            ff_dropout=dropout)
		# Project the the dimension of features from d_model into speaker nums.
		self.pred_layer = nn.Sequential(
			nn.Linear(d_model, d_model),
			nn.Sigmoid(),
			nn.Linear(d_model, n_spks),
		)

	def forward(self, mels):
		"""
		args:
			mels: (batch size, length, 40)
		return:
			out: (batch size, n_spks)
		"""
		# input: [B, seq_len, 40]
		out = self.prenet(mels)  # [B, seq_len, d_model]
		out = out.permute(1, 0, 2)  # [seq_len, B, d_model]
		# out = self.encoder_layer(out)  # [seq_len, B, d_model]
		out = self.conformer(out)
		out = out.transpose(0, 1)  # [B, seq_len, d_model]
		# mean pooling
		stats = out.mean(dim=1)  # [B, d_model]
		out = self.pred_layer(stats)  # [B, n_spks]
		return out

summary(ClassifierStrong(), (1, 100, 40))

Layer (type:depth-idx)                        Output Shape              Param #
ClassifierStrong                              [1, 600]                  --
├─Linear: 1-1                                 [1, 100, 512]             20,992
├─ConformerBlock: 1-2                         [100, 1, 512]             --
│    └─Scale: 2-1                             [100, 1, 512]             --
│    │    └─PreNorm: 3-1                      [100, 1, 512]             2,100,736
│    └─PreNorm: 2-2                           [100, 1, 512]             --
│    │    └─LayerNorm: 3-2                    [100, 1, 512]             1,024
│    │    └─Attention: 3-3                    [100, 1, 512]             1,114,688
│    └─ConformerConvModule: 2-3               [100, 1, 512]             --
│    │    └─Sequential: 3-4                   [100, 1, 512]             1,611,264
│    └─Scale: 2-4                             [100, 1, 512]             --
│    │    └─PreNorm: 3-5                      [100, 1, 512]        

# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [8]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
	optimizer: Optimizer,
	num_warmup_steps: int,
	num_training_steps: int,
	num_cycles: float = 0.5,
	last_epoch: int = -1,
):
	"""
	Create a schedule with a learning rate that decreases following the values of the cosine function between the
	initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
	initial lr set in the optimizer.

	Args:
		optimizer (:class:`~torch.optim.Optimizer`):
		The optimizer for which to schedule the learning rate.
		num_warmup_steps (:obj:`int`):
		The number of steps for the warmup phase.
		num_training_steps (:obj:`int`):
		The total number of training steps.
		num_cycles (:obj:`float`, `optional`, defaults to 0.5):
		The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
		following a half-cosine).
		last_epoch (:obj:`int`, `optional`, defaults to -1):
		The index of the last epoch when resuming training.

	Return:
		:obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
	"""
	def lr_lambda(current_step):
		# Warmup
		if current_step < num_warmup_steps:
			return float(current_step) / float(max(1, num_warmup_steps))
		# decadence
		progress = float(current_step - num_warmup_steps) / float(
			max(1, num_training_steps - num_warmup_steps)
		)
		return max(
			0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
		)

	return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [9]:
import torch


def model_fn(batch, model, criterion, device):
	"""Forward a batch through the model."""

	mels, labels = batch
	mels = mels.to(device)
	labels = labels.to(device)

	outs = model(mels)

	loss = criterion(outs, labels)

	# Get the speaker id with highest probability.
	preds = outs.argmax(1)
	# Compute accuracy.
	accuracy = torch.mean((preds == labels).float())

	return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [10]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device): 
	"""Validate on validation set."""

	model.eval()
	running_loss = 0.0
	running_accuracy = 0.0
	pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

	for i, batch in enumerate(dataloader):
		with torch.no_grad():
			loss, accuracy = model_fn(batch, model, criterion, device)
			running_loss += loss.item()
			running_accuracy += accuracy.item()

		pbar.update(dataloader.batch_size)
		pbar.set_postfix(
			loss=f"{running_loss / (i+1):.2f}",
			accuracy=f"{running_accuracy / (i+1):.2f}",
		)

	pbar.close()
	model.train()

	return running_accuracy / len(dataloader)

# Main function

In [11]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
	"""arguments"""
	config = {
		"data_dir": "data/Dataset",
		"save_path": "model.ckpt",
		"batch_size": 256,
		"n_workers": 8,
		"valid_steps": 2000,
		"warmup_steps": 1000,
		"save_steps": 10000,
		"total_steps": 100000,
  		"lr": 1e-4,
		"weight_decay": 1e-4,
	}

	return config


def main(
	data_dir,
	save_path,
	batch_size,
	n_workers,
	valid_steps,
	warmup_steps,
	total_steps,
	save_steps,
	lr,
 	weight_decay,
 	model
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")
	train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
	train_iterator = iter(train_loader)
	print(f"[Info]: Finish loading data!",flush = True)

	# model = Classifier(n_spks=speaker_num).to(device)
	# model = ClassifierMeidum(n_spks=speaker_num).to(device)
	criterion = nn.CrossEntropyLoss()
	optimizer = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
	scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
	print(f"[Info]: Finish creating model!",flush = True)

	best_accuracy = -1.0
	best_state_dict = None

	pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

	for step in range(total_steps):
		# Get data
		try:
			batch = next(train_iterator)
		except StopIteration:
			train_iterator = iter(train_loader)
			batch = next(train_iterator)

		loss, accuracy = model_fn(batch, model, criterion, device)
		batch_loss = loss.item()
		batch_accuracy = accuracy.item()

		# Updata model
		loss.backward()
		optimizer.step()
		scheduler.step()
		optimizer.zero_grad()

		# Log
		pbar.update()
		pbar.set_postfix(
			loss=f"{batch_loss:.2f}",
			accuracy=f"{batch_accuracy:.2f}",
			step=step + 1,
		)

		# Do validation
		if (step + 1) % valid_steps == 0:
			pbar.close()

			valid_accuracy = valid(valid_loader, model, criterion, device)

			# keep the best model
			if valid_accuracy > best_accuracy:
				best_accuracy = valid_accuracy
				best_state_dict = model.state_dict()

			pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

		# Save the best model so far.
		if (step + 1) % save_steps == 0 and best_state_dict is not None:
			torch.save(best_state_dict, save_path)
			pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

	pbar.close()


# if __name__ == "__main__":
# 	main(**parse_args())

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Classifier(n_spks=600).to(device)
config = parse_args()
config['save_path'] = 'model_simple.ckpt'
config['lr'] = 1e-3
config['total_steps'] = 70000
main(**config, model=model)

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [00:27<00:00, 73.65 step/s, accuracy=0.00, loss=5.86, step=2000] 
Valid:  99% 5632/5667 [00:00<00:00, 10025.89 uttr/s, accuracy=0.01, loss=5.87]
Train: 100% 2000/2000 [00:26<00:00, 74.98 step/s, accuracy=0.07, loss=5.35, step=4000] 
Valid:  99% 5632/5667 [00:00<00:00, 10693.26 uttr/s, accuracy=0.04, loss=5.39]
Train: 100% 2000/2000 [00:27<00:00, 73.42 step/s, accuracy=0.09, loss=5.06, step=6000] 
Valid:  99% 5632/5667 [00:00<00:00, 10698.28 uttr/s, accuracy=0.07, loss=5.03]
Train: 100% 2000/2000 [00:27<00:00, 73.22 step/s, accuracy=0.09, loss=4.76, step=8000] 
Valid:  99% 5632/5667 [00:00<00:00, 9899.35 uttr/s, accuracy=0.09, loss=4.76]
Train: 100% 2000/2000 [00:26<00:00, 74.46 step/s, accuracy=0.17, loss=4.45, step=1e+4] 
Valid:  99% 5632/5667 [00:00<00:00, 9853.57 uttr/s, accuracy=0.11, loss=4.57] 
Train:   1% 16/2000 [00:00<00:23, 83.99 step/s, accuracy=0.11, loss=4.59, step=1e+4]

Step 10000, best model saved. (accuracy=0.1135)


Train: 100% 2000/2000 [00:26<00:00, 74.87 step/s, accuracy=0.11, loss=4.38, step=12000] 
Valid:  99% 5632/5667 [00:00<00:00, 10577.45 uttr/s, accuracy=0.13, loss=4.40]
Train: 100% 2000/2000 [00:26<00:00, 74.80 step/s, accuracy=0.18, loss=4.15, step=14000] 
Valid:  99% 5632/5667 [00:00<00:00, 9242.96 uttr/s, accuracy=0.15, loss=4.26]
Train: 100% 2000/2000 [00:27<00:00, 73.64 step/s, accuracy=0.16, loss=4.13, step=16000] 
Valid:  99% 5632/5667 [00:00<00:00, 10031.83 uttr/s, accuracy=0.17, loss=4.14]
Train: 100% 2000/2000 [00:27<00:00, 73.91 step/s, accuracy=0.20, loss=4.00, step=18000] 
Valid:  99% 5632/5667 [00:00<00:00, 9639.94 uttr/s, accuracy=0.19, loss=4.02] 
Train: 100% 2000/2000 [00:26<00:00, 74.58 step/s, accuracy=0.25, loss=3.84, step=2e+4]  
Valid:  99% 5632/5667 [00:00<00:00, 9815.25 uttr/s, accuracy=0.20, loss=3.91] 
Train:   1% 23/2000 [00:00<00:18, 104.59 step/s, accuracy=0.26, loss=3.73, step=2e+4]

Step 20000, best model saved. (accuracy=0.2040)


...

Step 80000, best model saved. (accuracy=0.3816)


Train: 100% 2000/2000 [00:32<00:00, 61.68 step/s, accuracy=0.41, loss=2.82, step=82000]
Valid:  99% 5632/5667 [00:00<00:00, 11595.46 uttr/s, accuracy=0.38, loss=2.89]
Train: 100% 2000/2000 [00:32<00:00, 61.25 step/s, accuracy=0.45, loss=2.61, step=84000]
Valid:  99% 5632/5667 [00:00<00:00, 12471.21 uttr/s, accuracy=0.39, loss=2.88]
Train: 100% 2000/2000 [00:32<00:00, 61.41 step/s, accuracy=0.44, loss=2.47, step=86000]
Valid:  99% 5632/5667 [00:00<00:00, 12256.37 uttr/s, accuracy=0.37, loss=2.88]
Train: 100% 2000/2000 [00:32<00:00, 61.20 step/s, accuracy=0.36, loss=2.69, step=88000]
Valid:  99% 5632/5667 [00:00<00:00, 10667.71 uttr/s, accuracy=0.38, loss=2.88]
Train: 100% 2000/2000 [00:32<00:00, 61.01 step/s, accuracy=0.42, loss=2.65, step=9e+4] 
Valid:  99% 5632/5667 [00:00<00:00, 11860.35 uttr/s, accuracy=0.38, loss=2.88]
Train:   1% 13/2000 [00:00<00:28, 68.55 step/s, accuracy=0.43, loss=2.54, step=9e+4]

Step 90000, best model saved. (accuracy=0.3851)


Train: 100% 2000/2000 [00:32<00:00, 61.59 step/s, accuracy=0.41, loss=2.75, step=92000] 
Valid:  99% 5632/5667 [00:00<00:00, 12069.01 uttr/s, accuracy=0.39, loss=2.86]
Train: 100% 2000/2000 [00:33<00:00, 60.32 step/s, accuracy=0.44, loss=2.62, step=94000]
Valid:  99% 5632/5667 [00:00<00:00, 12359.42 uttr/s, accuracy=0.38, loss=2.86]
Train: 100% 2000/2000 [00:32<00:00, 61.32 step/s, accuracy=0.45, loss=2.59, step=96000]
Valid:  99% 5632/5667 [00:00<00:00, 11856.84 uttr/s, accuracy=0.38, loss=2.86]
Train: 100% 2000/2000 [00:32<00:00, 61.09 step/s, accuracy=0.41, loss=2.67, step=98000]
Valid:  99% 5632/5667 [00:00<00:00, 12027.98 uttr/s, accuracy=0.39, loss=2.86]
Train: 100% 2000/2000 [00:32<00:00, 61.54 step/s, accuracy=0.41, loss=2.71, step=1e+5] 
Valid:  99% 5632/5667 [00:00<00:00, 12039.17 uttr/s, accuracy=0.37, loss=2.89]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 100000, best model saved. (accuracy=0.3888)


In [13]:
config = parse_args()
config['save_path'] = 'model_medium.ckpt'
config['lr'] = 1e-4
config['total_steps'] = 25000
model = ClassifierMeidum(n_spks=600).to(device)
main(**config, model=model)

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [13:41<00:00,  2.44 step/s, accuracy=0.31, loss=3.66, step=2000]
Valid:  99% 5632/5667 [00:03<00:00, 1633.75 uttr/s, accuracy=0.27, loss=3.76]
Train: 100% 2000/2000 [15:30<00:00,  2.15 step/s, accuracy=0.62, loss=1.92, step=4000]
Valid:  99% 5632/5667 [00:03<00:00, 1627.75 uttr/s, accuracy=0.53, loss=2.29]
Train: 100% 2000/2000 [13:40<00:00,  2.44 step/s, accuracy=0.78, loss=1.07, step=6000]
Valid:  99% 5632/5667 [00:03<00:00, 1633.76 uttr/s, accuracy=0.66, loss=1.60]
Train: 100% 2000/2000 [13:40<00:00,  2.44 step/s, accuracy=0.83, loss=0.79, step=8000]
Valid:  99% 5632/5667 [00:03<00:00, 1638.21 uttr/s, accuracy=0.72, loss=1.28]
Train: 100% 2000/2000 [13:40<00:00,  2.44 step/s, accuracy=0.91, loss=0.47, step=1e+4]
Valid:  99% 5632/5667 [00:03<00:00, 1640.79 uttr/s, accuracy=0.75, loss=1.12]
Train:   0% 0/2000 [00:00<?, ? step/s]

Step 10000, best model saved. (accuracy=0.7495)


Train: 100% 2000/2000 [13:40<00:00,  2.44 step/s, accuracy=0.93, loss=0.37, step=12000]
Valid:  99% 5632/5667 [00:03<00:00, 1640.97 uttr/s, accuracy=0.77, loss=1.02]
Train: 100% 2000/2000 [13:40<00:00,  2.44 step/s, accuracy=0.95, loss=0.24, step=14000]
Valid:  99% 5632/5667 [00:03<00:00, 1643.02 uttr/s, accuracy=0.79, loss=0.95]
Train: 100% 2000/2000 [13:40<00:00,  2.44 step/s, accuracy=0.95, loss=0.21, step=16000]
Valid:  99% 5632/5667 [00:03<00:00, 1629.35 uttr/s, accuracy=0.80, loss=0.90]
Train: 100% 2000/2000 [13:40<00:00,  2.44 step/s, accuracy=0.98, loss=0.13, step=18000]
Valid:  99% 5632/5667 [00:03<00:00, 1641.45 uttr/s, accuracy=0.81, loss=0.91]
Train: 100% 2000/2000 [13:40<00:00,  2.44 step/s, accuracy=0.97, loss=0.14, step=2e+4] 
Valid:  99% 5632/5667 [00:03<00:00, 1644.28 uttr/s, accuracy=0.81, loss=0.89]
Train:   0% 0/2000 [00:00<?, ? step/s]

Step 20000, best model saved. (accuracy=0.8077)


Train: 100% 2000/2000 [13:40<00:00,  2.44 step/s, accuracy=0.96, loss=0.11, step=22000]
Valid:  99% 5632/5667 [00:03<00:00, 1635.75 uttr/s, accuracy=0.82, loss=0.86]
Train: 100% 2000/2000 [13:40<00:00,  2.44 step/s, accuracy=0.97, loss=0.15, step=24000]
Valid:  99% 5632/5667 [00:03<00:00, 1632.18 uttr/s, accuracy=0.83, loss=0.84]
Train: 100% 2000/2000 [07:41<00:00,  4.33 step/s, accuracy=0.98, loss=0.08, step=26000]
Valid:  99% 5632/5667 [00:01<00:00, 3130.88 uttr/s, accuracy=0.82, loss=0.87]
Train:  41% 825/2000 [02:39<03:45,  5.20 step/s, accuracy=0.99, loss=0.06, step=26825]

KeyboardInterrupt: 

In [14]:
config = parse_args()
config['save_path'] = 'model_strong.ckpt'
config['lr'] = 1e-4
config['total_steps'] = 100000
model = ClassifierStrong(n_spks=600).to(device)
main(**config, model=model)

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [03:57<00:00,  8.41 step/s, accuracy=0.12, loss=4.72, step=2000]
Valid:  99% 5632/5667 [00:01<00:00, 4419.23 uttr/s, accuracy=0.11, loss=4.76]
Train: 100% 2000/2000 [03:57<00:00,  8.41 step/s, accuracy=0.22, loss=3.84, step=4000]
Valid:  99% 5632/5667 [00:01<00:00, 4448.73 uttr/s, accuracy=0.23, loss=3.82]
Train: 100% 2000/2000 [03:57<00:00,  8.41 step/s, accuracy=0.33, loss=3.20, step=6000]
Valid:  99% 5632/5667 [00:01<00:00, 4460.21 uttr/s, accuracy=0.33, loss=3.22]
Train: 100% 2000/2000 [03:58<00:00,  8.40 step/s, accuracy=0.41, loss=2.78, step=8000]
Valid:  99% 5632/5667 [00:01<00:00, 4400.18 uttr/s, accuracy=0.40, loss=2.81]
Train: 100% 2000/2000 [03:57<00:00,  8.41 step/s, accuracy=0.57, loss=2.21, step=1e+4]
Valid:  99% 5632/5667 [00:01<00:00, 4459.88 uttr/s, accuracy=0.45, loss=2.50]
                                                                                      
Train:  41% 825/2000 [22:46<03:45,  5.20 step/s, accuracy=0.99, loss=0.06, step=26825]

Step 10000, best model saved. (accuracy=0.4538)


Train: 100% 2000/2000 [03:58<00:00,  8.40 step/s, accuracy=0.58, loss=1.95, step=12000]
Valid:  99% 5632/5667 [00:01<00:00, 4440.21 uttr/s, accuracy=0.50, loss=2.26]
Train: 100% 2000/2000 [03:58<00:00,  8.40 step/s, accuracy=0.64, loss=1.57, step=14000]
Valid:  99% 5632/5667 [00:01<00:00, 4443.42 uttr/s, accuracy=0.56, loss=2.03]
Train: 100% 2000/2000 [03:58<00:00,  8.40 step/s, accuracy=0.67, loss=1.48, step=16000]
Valid:  99% 5632/5667 [00:01<00:00, 4439.27 uttr/s, accuracy=0.58, loss=1.89]
Train: 100% 2000/2000 [03:58<00:00,  8.40 step/s, accuracy=0.70, loss=1.36, step=18000]
Valid:  99% 5632/5667 [00:01<00:00, 4459.22 uttr/s, accuracy=0.62, loss=1.72]
Train: 100% 2000/2000 [03:57<00:00,  8.41 step/s, accuracy=0.75, loss=1.23, step=2e+4]
Valid:  99% 5632/5667 [00:01<00:00, 4472.96 uttr/s, accuracy=0.66, loss=1.56]
                                                                                      
Train:  41% 825/2000 [42:43<03:45,  5.20 step/s, accuracy=0.99, loss=0.06, step=2682

Step 20000, best model saved. (accuracy=0.6555)


...

Step 80000, best model saved. (accuracy=0.7903)


Train: 100% 2000/2000 [03:57<00:00,  8.41 step/s, accuracy=1.00, loss=0.06, step=82000]
Valid:  99% 5632/5667 [00:01<00:00, 4529.33 uttr/s, accuracy=0.78, loss=1.03]
Train: 100% 2000/2000 [03:57<00:00,  8.41 step/s, accuracy=1.00, loss=0.03, step=84000]
Valid:  99% 5632/5667 [00:01<00:00, 4475.71 uttr/s, accuracy=0.79, loss=1.05]
Train: 100% 2000/2000 [03:58<00:00,  8.40 step/s, accuracy=1.00, loss=0.04, step=86000]
Valid:  99% 5632/5667 [00:01<00:00, 4500.24 uttr/s, accuracy=0.79, loss=1.03]
Train: 100% 2000/2000 [03:57<00:00,  8.40 step/s, accuracy=1.00, loss=0.04, step=88000]
Valid:  99% 5632/5667 [00:01<00:00, 4497.13 uttr/s, accuracy=0.78, loss=1.04]
Train: 100% 2000/2000 [03:57<00:00,  8.42 step/s, accuracy=1.00, loss=0.03, step=9e+4]
Valid:  99% 5632/5667 [00:01<00:00, 4500.47 uttr/s, accuracy=0.79, loss=1.01]
                                                                                        
Train:  41% 825/2000 [3:02:23<03:45,  5.20 step/s, accuracy=0.99, loss=0.06, step=

Step 90000, best model saved. (accuracy=0.7903)


Train: 100% 2000/2000 [03:57<00:00,  8.41 step/s, accuracy=1.00, loss=0.03, step=92000]
Valid:  99% 5632/5667 [00:01<00:00, 4530.40 uttr/s, accuracy=0.78, loss=1.04]
Train: 100% 2000/2000 [03:57<00:00,  8.41 step/s, accuracy=1.00, loss=0.03, step=94000]
Valid:  99% 5632/5667 [00:01<00:00, 4470.35 uttr/s, accuracy=0.78, loss=1.02]
Train: 100% 2000/2000 [03:57<00:00,  8.41 step/s, accuracy=1.00, loss=0.02, step=96000]
Valid:  99% 5632/5667 [00:01<00:00, 4501.41 uttr/s, accuracy=0.79, loss=1.04]
Train: 100% 2000/2000 [03:57<00:00,  8.41 step/s, accuracy=1.00, loss=0.03, step=98000]
Valid:  99% 5632/5667 [00:01<00:00, 4499.72 uttr/s, accuracy=0.78, loss=1.05]
Train: 100% 2000/2000 [03:57<00:00,  8.41 step/s, accuracy=1.00, loss=0.03, step=1e+5]
Valid:  99% 5632/5667 [00:01<00:00, 4526.14 uttr/s, accuracy=0.79, loss=1.02]
                                                                                        
Train:   0% 0/2000 [00:00<?, ? step/s]5.20 step/s, accuracy=0.99, loss=0.06, step=

Step 100000, best model saved. (accuracy=0.7905)


# Inference

## Dataset of inference

In [ ]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
	def __init__(self, data_dir):
		testdata_path = Path(data_dir) / "testdata.json"
		metadata = json.load(testdata_path.open())
		self.data_dir = data_dir
		self.data = metadata["utterances"]

	def __len__(self):
		return len(self.data)

	def __getitem__(self, index):
		utterance = self.data[index]
		feat_path = utterance["feature_path"]
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		return feat_path, mel


def inference_collate_batch(batch):
	"""Collate a batch of data."""
	feat_paths, mels = zip(*batch)

	return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [ ]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./data/Dataset",
		"model_path": "./model.ckpt",
		"output_path": "./output.csv",
	}

	return config


def main(
	data_dir,
	model_path,
	output_path,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	mapping_path = Path(data_dir) / "mapping.json"
	mapping = json.load(mapping_path.open())

	dataset = InferenceDataset(data_dir)
	dataloader = DataLoader(
		dataset,
		batch_size=1,
		shuffle=False,
		drop_last=False,
		num_workers=8,
		collate_fn=inference_collate_batch,
	)
	print(f"[Info]: Finish loading data!",flush = True)

	speaker_num = len(mapping["id2speaker"])
	model = Classifier(n_spks=speaker_num).to(device)
	model.load_state_dict(torch.load(model_path))
	model.eval()
	print(f"[Info]: Finish creating model!",flush = True)

	results = [["Id", "Category"]]
	for feat_paths, mels in tqdm(dataloader):
		with torch.no_grad():
			mels = mels.to(device)
			outs = model(mels)
			preds = outs.argmax(1).cpu().numpy()
			for feat_path, pred in zip(feat_paths, preds):
				results.append([feat_path, mapping["id2speaker"][str(pred)]])

	with open(output_path, 'w', newline='') as csvfile:
		writer = csv.writer(csvfile)
		writer.writerows(results)


if __name__ == "__main__":
	main(**parse_args())

In [ ]:
!rm -r Dataset 
#for the usage of "Save Version", only save the output needed to reduce the waiting time